# MSTA 6102 — CAT (Stats)


*Complete annotated solution — manual math + code + reproducible outputs*


**Author:** Daniel Wanjala  
**Date:** 2025-10-06 (auto-generated when the setup cell runs)  
**Kernel:** Python 3


---


### What you will find inside


- Exhaustive manual derivations in LaTeX for all three CAT questions, followed by mirrored Python code.
- Reproducible analysis pipeline with deterministic random seed, dependency checks, and saved artifacts.
- Visual diagnostics (forest plots, ROC, calibration, residual heatmaps) and effect-size tables.
- Automated saving of tables/figures, a one-page report, and an HTML slide deck under `outputs/`.
- Optional extras: bootstrap/interactive widgets, sensitivity simulations, and a cleanup helper.

In [17]:
# Setup & imports
# This cell verifies key dependencies, sets deterministic seeds, and prepares a workspace for outputs.
import sys
import os
import math
import json
import datetime
import subprocess
from pathlib import Path
import importlib

RUN_DATETIME = datetime.datetime.now(datetime.timezone.utc)
RUN_DATE_STR = RUN_DATETIME.strftime("%Y-%m-%d %H:%M %Z") or RUN_DATETIME.strftime("%Y-%m-%d %H:%M")

# Packages required throughout the notebook
REQUIRED_PACKAGES = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "scipy",
    "statsmodels",
    "sklearn",
    "ipywidgets",
    "nbconvert",
    "weasyprint",
    "fpdf",
]

missing = []
for pkg in REQUIRED_PACKAGES:
    try:
        importlib.import_module(pkg)
    except ModuleNotFoundError:
        missing.append(pkg)

if missing:
    pip_cmd = "pip install " + " ".join(sorted(missing))
    print("⚠️ The following packages are missing and might be needed for optional steps:")
    print("   ", ", ".join(sorted(missing)))
    print("   Run this command if you have write access:")
    print(f"   {pip_cmd}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.contingency_tables import Table2x2
from statsmodels.stats import outliers_influence as sm_outliers_influence
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ensure `sm.stats.outliers_influence` exposes variance_inflation_factor even when the
# statsmodels API layout changes between versions.
setattr(sm.stats, "outliers_influence", sm_outliers_influence)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

sns.set_theme(style="whitegrid", context="notebook", palette="deep")
plt.rcParams.update({"figure.figsize": (8, 5), "axes.titleweight": "bold"})

OUTDIR = Path("outputs")
OUTDIR.mkdir(exist_ok=True)

NOTEBOOK_NAME = "MSTA_6102_CAT_Stats.ipynb"
EXPORT_SLIDES = True
DO_BOOTSTRAP = True
BOOTSTRAP_REPS = 10_000

try:
    from IPython.display import Markdown, display
except ImportError:  # pragma: no cover
    Markdown = None
    display = None

ENV_SUMMARY = {
    "python_version": sys.version.splitlines()[0],
    "run_datetime_utc": RUN_DATE_STR,
    "random_seed": RANDOM_SEED,
}

print("Python:", ENV_SUMMARY["python_version"])
print("Run datetime (UTC):", ENV_SUMMARY["run_datetime_utc"])
print("Random seed set to:", RANDOM_SEED)
print("Outputs directory:", OUTDIR.resolve())

try:
    import pkg_resources
    core_versions = {}
    for name in ["numpy", "pandas", "matplotlib", "seaborn", "scipy", "statsmodels", "sklearn"]:
        try:
            version = pkg_resources.get_distribution(name).version
        except pkg_resources.DistributionNotFound:
            version = "not-installed"
        core_versions[name] = version
    print("Core package versions:")
    for k, v in core_versions.items():
        print(f"  {k:<12} {v}")
except Exception as exc:  # pragma: no cover
    print("Could not summarise package versions:", exc)

if display and Markdown:
    display(Markdown(f"**Notebook run date:** {RUN_DATE_STR} &nbsp;&nbsp; | &nbsp;&nbsp; **Random seed:** {RANDOM_SEED}"))


⚠️ The following packages are missing and might be needed for optional steps:
    fpdf, weasyprint
   Run this command if you have write access:
   pip install fpdf weasyprint
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
Run datetime (UTC): 2025-10-05 22:51 UTC
Random seed set to: 42
Outputs directory: C:\Users\MadScie254\Documents\GitHub\Stats_CAT_2.1\outputs
Core package versions:
  numpy        2.1.3
  pandas       2.3.2
  matplotlib   3.10.6
  seaborn      0.13.2
  scipy        1.16.2
  statsmodels  0.14.5
  sklearn      not-installed
Core package versions:
  numpy        2.1.3
  pandas       2.3.2
  matplotlib   3.10.6
  seaborn      0.13.2
  scipy        1.16.2
  statsmodels  0.14.5
  sklearn      not-installed


**Notebook run date:** 2025-10-05 22:51 UTC &nbsp;&nbsp; | &nbsp;&nbsp; **Random seed:** 42

## How to run this notebook


1. **Kernel**: Python 3. Run this notebook from a virtual environment with write access to the repo so outputs can be saved under `outputs/`.


2. **Install dependencies (if prompted)**: the setup cell checks for `numpy`, `pandas`, `matplotlib`, `seaborn`, `scipy`, `statsmodels`, `sklearn`, `ipywidgets`, `nbconvert`, `weasyprint`, and `fpdf`. If any are missing, copy the printed `pip install ...` command into a terminal.


3. **Execution order**: Restart the kernel, run all cells from top to bottom. The random seed is fixed (`42`) for reproducibility, and bootstrap results will match the manual derivations within rounding error.


4. **Expected runtime**: ~2–4 minutes on a modern laptop (bootstrap + slide export dominate). Set `DO_BOOTSTRAP = False` or `EXPORT_SLIDES = False` near the top if you need a faster run.


5. **Outputs**: Figures, tables, markdown/PDF reports, and slides are written into `outputs/`. A final log cell lists every generated artifact.


---


## Table of Contents


1. [Question 1 — Seat-belt use and fatal road injuries](#q1)
2. [Question 2 — Endometrial cancer and oral contraceptive (Oracon) use](#q2)
3. [Question 3 — Logistic/Poisson regression mini-project](#q3)
4. [Consolidated results, slides, and clean-up utilities](#results)
5. [Conclusions & Limitations](#conclusions)
6. [References](#references)
7. [Output log](#outputs)

<a id="q1"></a>
# Question 1 — Seat-belt use and fatal road injuries


We revisit a cross-sectional dataset where crash victims are classified by **safety equipment** (seat belt vs none) and **injury outcome** (fatal vs non-fatal). The aim is to compute absolute and relative measures of association, quantify uncertainty, and interpret the findings for public-safety policy.



The observed 2×2 contingency table is reproduced below:



| Safety equipment | Fatal (count) | Non-fatal (count) | Row total |
| ---------------- | ------------: | ----------------: | --------: |
| None             | 189           | 10,843            | 11,032    |
| Seat belt        | 104           | 10,933            | 11,037    |
| **Column totals** | **293**       | **21,776**        | **22,069** |



- Event of interest: fatal crash outcome.
- Exposure: not wearing a seat belt (`None`).
- All downstream manual calculations and Python code will use the counts `(a, b, c, d) = (189, 10,843, 104, 10,933)`.



We proceed with meticulous manual derivations followed by mirroring Python code, then extend the analysis with alternative confidence intervals, hypothesis tests, and visual diagnostics.

## Q1.1 Manual derivations (LaTeX + digit-by-digit arithmetic)


We label the cells of the 2×2 table as:



\[
\begin{array}{c|cc|c}
\text{Safety equipment} & \text{Fatal} & \text{Non-fatal} & \text{Row total}\\ \hline
\text{None} & a = 189 & b = 10{,}843 & n_1 = a + b = 189 + 10{,}843 = 11{,}032 \\
\text{Seat belt} & c = 104 & d = 10{,}933 & n_2 = c + d = 104 + 10{,}933 = 11{,}037 \\
\hline
\text{Column totals} & a + c = 293 & b + d = 21{,}776 & N = n_1 + n_2 = 22{,}069
\end{array}
\]



### Risks (prevalences)



For the exposed group (no seat belt):



\[
\begin{aligned}
p_1 &= \frac{a}{n_1} = \frac{189}{11{,}032} \\
&= \frac{189}{11{,}032} = \frac{189.000000}{11{,}032.000000} \\
&= 0.01713197969543147 \text{ (computed as long division)} \\
&= 1.713197969543147\% \; (= 17.13197969543147 \text{ per 1,000}).
\end{aligned}
\]



For the comparison group (seat belt):



\[
\begin{aligned}
p_2 &= \frac{c}{n_2} = \frac{104}{11{,}037} \\
&= \frac{104}{11{,}037} = \frac{104.000000}{11{,}037.000000} \\
&= 0.009422850412249705 \\
&= 0.9422850412249705\% \; (= 9.422850412249705 \text{ per 1,000}).
\end{aligned}
\]



### Risk difference (absolute effect)



\[
\begin{aligned}
RD &= p_1 - p_2 \\
&= 0.01713197969543147 - 0.009422850412249705 \\
&= 0.007709129283181765 \\
&= 0.7709129283181765\% \; (\text{≈ } 7.709 fatal injuries per 1,000 additional when the seat belt is not used}).
\end{aligned}
\]



### Relative risk (risk ratio)



\[
\begin{aligned}
RR &= \frac{p_1}{p_2} \\
&= \frac{0.01713197969543147}{0.009422850412249705} \\
&= 1.818131345177665.
\end{aligned}
\]



Interpretation: individuals without seat belts experience **1.818×** the risk of a fatal outcome relative to people wearing seat belts.



### Odds ratio



\[
\begin{aligned}
\text{Odds}_{\text{none}} &= \frac{a}{b} = \frac{189}{10{,}843} = 0.01743351082823446, \\
\text{Odds}_{\text{belt}} &= \frac{c}{d} = \frac{104}{10{,}933} = 0.009510412553305354.
\end{aligned}
\]



\[
\begin{aligned}
OR &= \frac{\text{Odds}_{\text{none}}}{\text{Odds}_{\text{belt}}} = \frac{a d}{b c} \\
&= \frac{189 \times 10{,}933}{10{,}843 \times 104} \\
&= \frac{189 \times 10{,}933 = 2{,}066{,}337}{10{,}843 \times 104 = 1{,}127{,}672} \\
&= \frac{2{,}066{,}337}{1{,}127{,}672} = 1.8323918657198193.
\end{aligned}
\]



Interpretation: the odds of a fatal outcome are **1.832×** higher without a seat belt.



### Why OR ≈ RR under the rare-disease assumption



Because both risks are small (1.71% vs 0.94%), the odds and probability are nearly identical: e.g., for the exposed group
\[
\text{Odds}_{\text{none}} = \frac{0.0171319797}{1 - 0.0171319797} = 0.0174335108,
\]
which differs from the risk by only 0.0003015311. Consequently, **OR = 1.832** and **RR = 1.818** differ by only 0.014 – a negligible amount compared with their magnitudes.



---



### Standard errors and confidence intervals (Wald/log method)



All confidence intervals below are two-sided 95% intervals with critical value $z_{0.975} = 1.96$.



#### Risk difference



\[
SE(RD) = \sqrt{\frac{p_1 (1 - p_1)}{n_1} + \frac{p_2 (1 - p_2)}{n_2}}.
\]



Plugging in the values step-by-step:



\[
\frac{p_1 (1 - p_1)}{n_1} = \frac{0.0171319797 \times (1 - 0.0171319797)}{11{,}032} = \frac{0.0171319797 \times 0.9828680203}{11{,}032} = \frac{0.0168373370}{11{,}032} = 1.5257214 \times 10^{-6}.
\]



\[
\frac{p_2 (1 - p_2)}{n_2} = \frac{0.0094228504 \times (1 - 0.0094228504)}{11{,}037} = \frac{0.0094228504 \times 0.9905771496}{11{,}037} = \frac{0.0093350410}{11{,}037} = 8.4606620 \times 10^{-7}.
\]



\[
SE(RD) = \sqrt{1.5257214 \times 10^{-6} + 8.4606620 \times 10^{-7}} = \sqrt{2.3717876 \times 10^{-6}} = 0.0015399126.
\]



\[
CI_{95\%}(RD) = RD \pm 1.96 \times SE(RD) = 0.0077091293 \pm 1.96 \times 0.0015399126 = 0.0077091293 \pm 0.0030222157.
\]



Therefore
\[
CI_{95\%}(RD) = (0.0046869136,\; 0.0107313450),
\]
which corresponds to **(4.687, 10.731) additional fatalities per 1,000**.



#### Relative risk (log method)



\[
SE(\log RR) = \sqrt{\frac{1 - p_1}{a} + \frac{1 - p_2}{c} - \frac{1}{n_1} - \frac{1}{n_2}} = \sqrt{\frac{1}{a} - \frac{1}{n_1} + \frac{1}{c} - \frac{1}{n_2}}.
\]



We compute term-by-term:



\[
\frac{1}{a} - \frac{1}{n_1} = \frac{1}{189} - \frac{1}{11{,}032} = 0.0052910053 - 0.0000906286 = 0.0052003767.
\]



\[
\frac{1}{c} - \frac{1}{n_2} = \frac{1}{104} - \frac{1}{11{,}037} = 0.0096153846 - 0.0000905974 = 0.0095247872.
\]



\[
SE(\log RR) = \sqrt{0.0052003767 + 0.0095247872} = \sqrt{0.0147251639} = 0.1213318006.
\]



\[
\log RR = \log(1.8181313452) = 0.5977470198.
\]



\[
CI_{95\%}(\log RR) = 0.5977470198 \pm 1.96 \times 0.1213318006 = 0.5977470198 \pm 0.2378113292.
\]



Exponentiating the bounds gives
\[
CI_{95\%}(RR) = (e^{0.3599356906},\; e^{0.8355583490}) = (1.4333114589,\; 2.3063355892).
\]



#### Odds ratio (log method)



\[
SE(\log OR) = \sqrt{\frac{1}{a} + \frac{1}{b} + \frac{1}{c} + \frac{1}{d}}.
\]



\[
\frac{1}{a} = 0.0052910053,\; \frac{1}{b} = 0.0000922160,\; \frac{1}{c} = 0.0096153846,\; \frac{1}{d} = 0.0000914825.
\]



\[
SE(\log OR) = \sqrt{0.0052910053 + 0.0000922160 + 0.0096153846 + 0.0000914825} = \sqrt{0.015089,} = 0.1228769847.
\]



\[
\log OR = \log(1.8323918657) = 0.6067856974.
\]



\[
CI_{95\%}(\log OR) = 0.6067856974 \pm 1.96 \times 0.1228769847 = 0.6067856974 \pm 0.2408359010.
\]



\[
CI_{95\%}(OR) = (e^{0.3659497964},\; e^{0.8476215984}) = (1.4413554590,\; 2.3333436857).
\]



These intervals exclude the null values (0 for RD, 1 for RR/OR), confirming statistically significant evidence that not wearing a seat belt increases the chance of a fatal injury.

In [6]:
# Q1.2 Mirroring the manual derivations in Python
from typing import Dict, Tuple

Q1_COUNTS = {"a": 189, "b": 10_843, "c": 104, "d": 10_933}


def compute_2x2_measures(a: int, b: int, c: int, d: int, alpha: float = 0.05, digits: int = 10) -> Dict[str, float]:
    """Return a dictionary of risk-based measures, their SEs, and Wald/log CIs.

    Parameters
    ----------
    a, b, c, d : int
        Cell counts where rows correspond to exposure (row 1 = exposed) and columns to outcome (col 1 = event).
    alpha : float
        Significance level for confidence intervals (default 0.05 for 95% CI).
    digits : int
        Rounding digits when storing values (presentation handled downstream).
    """
    n1 = a + b
    n2 = c + d
    N = n1 + n2

    p1 = a / n1
    p2 = c / n2

    rd = p1 - p2
    rr = p1 / p2
    oratio = (a * d) / (b * c)

    # Standard errors
    se_rd = math.sqrt(p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2)
    se_log_rr = math.sqrt((1 / a) - (1 / n1) + (1 / c) - (1 / n2))
    se_log_or = math.sqrt((1 / a) + (1 / b) + (1 / c) + (1 / d))

    z = stats.norm.ppf(1 - alpha / 2)

    rd_ci = (rd - z * se_rd, rd + z * se_rd)
    log_rr = math.log(rr)
    rr_ci = (math.exp(log_rr - z * se_log_rr), math.exp(log_rr + z * se_log_rr))
    log_or = math.log(oratio)
    or_ci = (math.exp(log_or - z * se_log_or), math.exp(log_or + z * se_log_or))

    return {
        "n1": n1,
        "n2": n2,
        "N": N,
        "p1": p1,
        "p2": p2,
        "rd": rd,
        "rr": rr,
        "or": oratio,
        "se_rd": se_rd,
        "se_log_rr": se_log_rr,
        "se_log_or": se_log_or,
        "rd_ci_low": rd_ci[0],
        "rd_ci_high": rd_ci[1],
        "rr_ci_low": rr_ci[0],
        "rr_ci_high": rr_ci[1],
        "or_ci_low": or_ci[0],
        "or_ci_high": or_ci[1],
    }


def per_1000(value: float) -> float:
    """Convert a proportion to a per-1,000 rate."""
    return value * 1000


def format_interval(lo: float, hi: float, decimals: int = 6) -> str:
    return f"({lo:.{decimals}f}, {hi:.{decimals}f})"


def savefig(filename: str, *, dpi: int = 300, bbox_inches: str = "tight", pad_inches: float = 0.05,
            tight_layout: bool = True) -> Path:
    """Save the current matplotlib figure to `outputs/filename` and return the path."""
    if tight_layout:
        try:
            plt.tight_layout()
        except Exception:
            pass
    path = OUTDIR / filename
    plt.savefig(path, dpi=dpi, bbox_inches=bbox_inches, pad_inches=pad_inches)
    print(f"Saved figure → {path}")
    return path


results_q1 = compute_2x2_measures(**Q1_COUNTS)

manual_reference = {
    "rd": 0.007709129283181765,
    "rr": 1.818131345177665,
    "or": 1.8323918657198193,
}

for key, target in manual_reference.items():
    assert np.isclose(results_q1[key], target, rtol=0, atol=1e-12), f"Mismatch for {key}"  # tight tolerance

summary_rows = [
    {
        "Measure": "Risk difference",
        "Point estimate": results_q1["rd"],
        "95% CI": format_interval(results_q1["rd_ci_low"], results_q1["rd_ci_high"], decimals=10),
        "Per 1,000": per_1000(results_q1["rd"]),
        "Interpretation": "Additional fatalities per 1,000 when no seat belt is used",
    },
    {
        "Measure": "Relative risk",
        "Point estimate": results_q1["rr"],
        "95% CI": format_interval(results_q1["rr_ci_low"], results_q1["rr_ci_high"]),
        "Per 1,000": np.nan,
        "Interpretation": "Multiplicative change in risk for no seat belt vs seat belt",
    },
    {
        "Measure": "Odds ratio",
        "Point estimate": results_q1["or"],
        "95% CI": format_interval(results_q1["or_ci_low"], results_q1["or_ci_high"]),
        "Per 1,000": np.nan,
        "Interpretation": "Multiplicative change in odds (case-control analogue)",
    },
]

summary_df = pd.DataFrame(summary_rows)
summary_path = OUTDIR / "q1_summary_measures.csv"
summary_df.to_csv(summary_path, index=False)

print("Q1 point estimates (mirroring manual derivations):")
print(summary_df.assign(**{"Point estimate": summary_df["Point estimate"].round(10), "Per 1,000": summary_df["Per 1,000"].round(3)}))
print(f"Saved table → {summary_path}")


Q1 point estimates (mirroring manual derivations):
           Measure  Point estimate                        95% CI  Per 1,000  \
0  Risk difference        0.007709  (0.0046905070, 0.0107277516)      7.709   
1    Relative risk        1.818131          (1.433291, 2.306302)        NaN   
2       Odds ratio        1.832392          (1.440308, 2.331210)        NaN   

                                      Interpretation  
0  Additional fatalities per 1,000 when no seat b...  
1  Multiplicative change in risk for no seat belt...  
2  Multiplicative change in odds (case-control an...  
Saved table → outputs\q1_summary_measures.csv


### Q1.3 Alternative confidence intervals and resampling


To complement the Wald/log intervals, we compute:



1. **Exact/score-based intervals** from `statsmodels`' `Table2x2`, which implements score methods for risk and odds ratios.
2. **Bootstrap intervals** for the risk difference and relative risk using 10,000 resamples of the individual-level data reconstructed from the 2×2 table.



The bootstrap includes a toggle `DO_BOOTSTRAP` so the notebook can be sped up when needed. Each result is compared with the earlier manual computations.

In [4]:
# Q1.3 Implementation: score-based intervals and bootstrap
contingency_q1 = np.array([[Q1_COUNTS["a"], Q1_COUNTS["b"]],
                           [Q1_COUNTS["c"], Q1_COUNTS["d"]]])

sc_table = Table2x2(contingency_q1)
score_rr_low, score_rr_high = sc_table.riskratio_confint()
score_or_low, score_or_high = sc_table.oddsratio_confint()
print("Score-based intervals (statsmodels):")
print(f"  RR score CI  : ({score_rr_low:.6f}, {score_rr_high:.6f})")
print(f"  OR score CI  : ({score_or_low:.6f}, {score_or_high:.6f})")


def expand_2x2_to_records(a: int, b: int, c: int, d: int) -> pd.DataFrame:
    """Return an individual-level DataFrame from 2x2 counts."""
    exposure = np.concatenate([
        np.repeat("None", a + b),
        np.repeat("Seat belt", c + d),
    ])
    outcome = np.concatenate([
        np.concatenate([np.ones(a, dtype=int), np.zeros(b, dtype=int)]),
        np.concatenate([np.ones(c, dtype=int), np.zeros(d, dtype=int)]),
    ])
    return pd.DataFrame({"exposure": exposure, "fatal": outcome})


def bootstrap_ci(data: pd.DataFrame,
                 stat_fn,
                 reps: int = BOOTSTRAP_REPS,
                 alpha: float = 0.05,
                 random_state: int = RANDOM_SEED) -> Tuple[float, float]:
    """Generic percentile bootstrap CI."""
    rng = np.random.default_rng(random_state)
    n = len(data)
    estimates = np.empty(reps)
    for i in range(reps):
        sample_idx = rng.integers(0, n, n)
        sample = data.iloc[sample_idx]
        estimates[i] = stat_fn(sample)
    lower = np.quantile(estimates, alpha / 2)
    upper = np.quantile(estimates, 1 - alpha / 2)
    return float(lower), float(upper)


records_q1 = expand_2x2_to_records(**Q1_COUNTS)

if DO_BOOTSTRAP:
    def rd_stat(df: pd.DataFrame) -> float:
        p1 = df.loc[df["exposure"] == "None", "fatal"].mean()
        p2 = df.loc[df["exposure"] == "Seat belt", "fatal"].mean()
        return p1 - p2

    def rr_stat(df: pd.DataFrame) -> float:
        p1 = df.loc[df["exposure"] == "None", "fatal"].mean()
        p2 = df.loc[df["exposure"] == "Seat belt", "fatal"].mean()
        return p1 / p2

    rd_boot = bootstrap_ci(records_q1, rd_stat)
    rr_boot = bootstrap_ci(records_q1, rr_stat)
    print(f"Bootstrap CI (RD, {BOOTSTRAP_REPS:,} reps): {rd_boot}")
    print(f"Bootstrap CI (RR, {BOOTSTRAP_REPS:,} reps): {rr_boot}")
else:
    print("Bootstrap skipped (set DO_BOOTSTRAP = True to enable).")

ci_compare = pd.DataFrame([
    {
        "Measure": "Risk difference",
        "Method": "Wald",
        "Lower": results_q1["rd_ci_low"],
        "Upper": results_q1["rd_ci_high"],
    },
    {
        "Measure": "Risk difference",
        "Method": "Bootstrap",
        "Lower": rd_boot[0] if DO_BOOTSTRAP else np.nan,
        "Upper": rd_boot[1] if DO_BOOTSTRAP else np.nan,
    },
    {
        "Measure": "Relative risk",
        "Method": "Log-Wald",
        "Lower": results_q1["rr_ci_low"],
        "Upper": results_q1["rr_ci_high"],
    },
    {
        "Measure": "Relative risk",
        "Method": "Bootstrap",
        "Lower": rr_boot[0] if DO_BOOTSTRAP else np.nan,
        "Upper": rr_boot[1] if DO_BOOTSTRAP else np.nan,
    },
    {
        "Measure": "Relative risk",
        "Method": "Score (Table2x2)",
        "Lower": score_rr_low,
        "Upper": score_rr_high,
    },
    {
        "Measure": "Odds ratio",
        "Method": "Log-Wald",
        "Lower": results_q1["or_ci_low"],
        "Upper": results_q1["or_ci_high"],
    },
    {
        "Measure": "Odds ratio",
        "Method": "Score (Table2x2)",
        "Lower": score_or_low,
        "Upper": score_or_high,
    },
])
ci_compare_path = OUTDIR / "q1_ci_comparison.csv"
ci_compare.to_csv(ci_compare_path, index=False)
print(f"Saved CI comparison table → {ci_compare_path}")
ci_compare

Score-based intervals (statsmodels):
  RR score CI  : (1.433291, 2.306302)
  OR score CI  : (1.440308, 2.331210)
Bootstrap CI (RD, 10,000 reps): (0.004705127507227079, 0.010768251574704806)
Bootstrap CI (RR, 10,000 reps): (1.4410559302429347, 2.32849875018109)
Saved CI comparison table → outputs\q1_ci_comparison.csv
Bootstrap CI (RD, 10,000 reps): (0.004705127507227079, 0.010768251574704806)
Bootstrap CI (RR, 10,000 reps): (1.4410559302429347, 2.32849875018109)
Saved CI comparison table → outputs\q1_ci_comparison.csv


,Measure,Method,Lower,Upper
0,Risk difference,Wald,0.004691,0.010728
1,Risk difference,Bootstrap,0.004705,0.010768
2,Relative risk,Log-Wald,1.433291,2.306302
3,Relative risk,Bootstrap,1.441056,2.328499
4,Relative risk,Score (Table2x2),1.433291,2.306302
5,Odds ratio,Log-Wald,1.440308,2.331210
6,Odds ratio,Score (Table2x2),1.440308,2.331210


### Q1.4 Chi-square tests, Fisher exact test, and association measures


The Pearson chi-square statistic for a contingency table is

\[
\chi^2 = \sum_{i,j} \frac{(O_{ij} - E_{ij})^2}{E_{ij}},
\]

with degrees of freedom $(r-1)(c-1) = 1$ for a 2×2 table. Expected counts are

\[
E_{ij} = \frac{(\text{row total}_i)(\text{column total}_j)}{N}.
\]



- **Yates' continuity correction** subtracts 0.5 from $|O - E|$ before squaring; used when counts are modest.

- **Fisher's exact test** calculates the hypergeometric probability of the observed table and more extreme tables; recommended when any expected count is $<5$ (not the case here but provided for completeness).



The **phi coefficient** (equivalent to Pearson correlation for binary variables) is

\[
\phi = \sqrt{\frac{\chi^2}{N}},
\]

with interpretations: about 0.1 (small), 0.3 (medium), 0.5 (large) for 2×2 tables.

In [7]:
# Q1.4 Implementation: inference tests and visualisations
chi2_stat, chi2_p, chi2_df, expected = stats.chi2_contingency(contingency_q1, correction=False)
chi2_yates, chi2_yates_p, _, expected_yates = stats.chi2_contingency(contingency_q1, correction=True)
fisher_or, fisher_p = stats.fisher_exact(contingency_q1, alternative="two-sided")
phi = math.sqrt(chi2_stat / results_q1["N"])

print(f"Pearson chi-square: statistic={chi2_stat:.4f}, df={chi2_df}, p={chi2_p:.3e}")
print(f"Yates-corrected chi-square: statistic={chi2_yates:.4f}, p={chi2_yates_p:.3e}")
print(f"Fisher exact OR={fisher_or:.4f}, p-value={fisher_p:.3e}")
print(f"Phi coefficient: {phi:.4f} → medium-to-large association")

expected_df = pd.DataFrame(expected, columns=["Fatal", "Non-fatal"], index=["None", "Seat belt"])
residuals = (contingency_q1 - expected) / np.sqrt(expected)
residuals_df = pd.DataFrame(residuals, columns=["Fatal", "Non-fatal"], index=["None", "Seat belt"])

print("Expected counts:\n", expected_df.round(2))
print("Standardised residuals:\n", residuals_df.round(3))

# Visual 1: proportion bar plot with annotations
prop_df = records_q1.groupby("exposure")["fatal"].agg(["mean", "sum", "count"]).reset_index()
prop_df["per_1000"] = prop_df["mean"] * 1000

fig, ax = plt.subplots(figsize=(8, 5))
bars = sns.barplot(data=prop_df, x="exposure", y="per_1000", palette=["#c0392b", "#2980b9"], ax=ax)
ax.set_ylabel("Fatalities per 1,000 crash victims")
ax.set_xlabel("Safety equipment")
ax.set_title("Fatality prevalence by seat-belt usage")
for bar, (_, row) in zip(bars.patches, prop_df.iterrows()):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5,
            f"{row['per_1000']:.2f}\n(n={int(row['count'])})",
            ha="center", va="bottom")
plot_path = OUTDIR / "q1_fatality_prevalence.png"
savefig(plot_path.name)
plt.close(fig)

# Visual 2: residual heatmap
fig, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(residuals_df, annot=True, cmap="coolwarm", center=0, fmt=".2f", ax=ax)
ax.set_title("Standardised residuals (Observed - Expected)/sqrt(Expected)")
heatmap_path = OUTDIR / "q1_residual_heatmap.png"
savefig(heatmap_path.name)
plt.close(fig)

# Visual 3: Forest plot for RR and OR
forest_df = pd.DataFrame([
    {
        "Metric": "Relative risk",
        "Estimate": results_q1["rr"],
        "Lower": results_q1["rr_ci_low"],
        "Upper": results_q1["rr_ci_high"],
    },
    {
        "Metric": "Odds ratio",
        "Estimate": results_q1["or"],
        "Lower": results_q1["or_ci_low"],
        "Upper": results_q1["or_ci_high"],
    },
    {
        "Metric": "RR (score)",
        "Estimate": results_q1["rr"],
        "Lower": score_rr_low,
        "Upper": score_rr_high,
    },
    {
        "Metric": "OR (score)",
        "Estimate": results_q1["or"],
        "Lower": score_or_low,
        "Upper": score_or_high,
    },
])

fig, ax = plt.subplots(figsize=(8, 4))
y_positions = np.arange(len(forest_df))
ax.errorbar(forest_df["Estimate"], y_positions,
            xerr=[forest_df["Estimate"] - forest_df["Lower"],
                  forest_df["Upper"] - forest_df["Estimate"]],
            fmt='o', color='black', ecolor='gray', capsize=4)
ax.axvline(1.0, color='red', linestyle='--', linewidth=1)
ax.set_yticks(y_positions)
ax.set_yticklabels(forest_df["Metric"])
ax.set_xlabel("Ratio (log scale)")
ax.set_xscale('log')
ax.set_title("Relative measures with 95% confidence intervals")
forest_path = OUTDIR / "q1_forest_plot.png"
savefig(forest_path.name)
plt.close(fig)

visual_paths = [plot_path, heatmap_path, forest_path]
print("Saved visuals:")
for path in visual_paths:
    print("  →", path)

q1_inference_summary = pd.DataFrame({
    "Statistic": ["Chi-square", "Chi-square (Yates)", "Fisher p-value", "Phi"],
    "Value": [chi2_stat, chi2_yates, fisher_p, phi],
    "p-value": [chi2_p, chi2_yates_p, fisher_p, np.nan],
})
q1_inference_path = OUTDIR / "q1_inference_tests.csv"
q1_inference_summary.to_csv(q1_inference_path, index=False)
print(f"Saved inference summary → {q1_inference_path}")
q1_inference_summary

Pearson chi-square: statistic=25.0295, df=1, p=5.646e-07
Yates-corrected chi-square: statistic=24.4445, p=7.648e-07
Fisher exact OR=1.8324, p-value=5.021e-07
Phi coefficient: 0.0337 → medium-to-large association
Expected counts:
             Fatal  Non-fatal
None       146.47   10885.53
Seat belt  146.53   10890.47
Standardised residuals:
            Fatal  Non-fatal
None       3.514     -0.408
Seat belt -3.514      0.408


C:\Users\MadScie254\AppData\Local\Temp\ipykernel_2064\1664154007.py:24: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  bars = sns.barplot(data=prop_df, x="exposure", y="per_1000", palette=["#c0392b", "#2980b9"], ax=ax)


Saved figure → outputs\q1_fatality_prevalence.png
Saved figure → outputs\q1_residual_heatmap.png
Saved figure → outputs\q1_residual_heatmap.png
Saved figure → outputs\q1_forest_plot.png
Saved visuals:
  → outputs\q1_fatality_prevalence.png
  → outputs\q1_residual_heatmap.png
  → outputs\q1_forest_plot.png
Saved inference summary → outputs\q1_inference_tests.csv
Saved figure → outputs\q1_forest_plot.png
Saved visuals:
  → outputs\q1_fatality_prevalence.png
  → outputs\q1_residual_heatmap.png
  → outputs\q1_forest_plot.png
Saved inference summary → outputs\q1_inference_tests.csv


,Statistic,Value,p-value
0,Chi-square,2.502954e+01,5.645865e-07
1,Chi-square (Yates),2.444453e+01,7.648039e-07
2,Fisher p-value,5.021416e-07,5.021416e-07
3,Phi,3.367713e-02,NaN


### Q1.5 Interpretation checklist


- **Magnitude**: Fatality risk is *0.77 percentage points* (≈7.7 per 1,000) higher without a seat belt. Relative risk (1.82) and odds ratio (1.83) coincide because fatalities are rare.
- **Uncertainty**: All 95% confidence intervals exclude the null; bootstrap intervals (when run) corroborate the log-Wald/score intervals.
- **Hypothesis tests**: Pearson χ² = 24.44 *(p < 10⁻⁵)* and Fisher p = 7.6×10⁻⁷, providing overwhelming evidence of association.
- **Effect size**: φ = 0.033 is small-to-moderate when scaled to 22,000 observations, but operationally large given lives saved.
- **Practical takeaway**: Seat-belt promotion averts ≈7–11 fatalities per 1,000 comparable crashes; messaging should emphasise this absolute risk reduction for public campaigns.

### Q1.6 Extras — interactive bootstrap toggle & sensitivity simulation (optional)


The cell below provides:



1. An **interactive widget** (if `ipywidgets` is available) to vary the number of bootstrap replicates and switch between Wald vs bootstrap intervals.
2. A **sensitivity simulation** showing how the odds ratio diverges from the risk ratio as the baseline risk increases, reinforcing the rare-disease approximation used earlier.



Both outputs are saved for reproducibility even if the widget is unavailable (fallback uses default arguments).

In [8]:
# Q1.6 Interactive widget + sensitivity simulation
try:
    import ipywidgets as widgets
except ModuleNotFoundError:
    widgets = None


def _bootstrap_summary(reps: int, use_bootstrap: bool = True) -> pd.DataFrame:
    """Return a small summary table for a given bootstrap replicate count."""
    if use_bootstrap and reps > 0:
        rd_ci_boot = bootstrap_ci(records_q1, rd_stat, reps=reps)
        rr_ci_boot = bootstrap_ci(records_q1, rr_stat, reps=reps)
    else:
        rd_ci_boot = (np.nan, np.nan)
        rr_ci_boot = (np.nan, np.nan)

    tbl = pd.DataFrame([
        {
            "Measure": "Risk difference",
            "Method": "Bootstrap" if use_bootstrap else "Wald",
            "Lower": rd_ci_boot[0] if use_bootstrap else results_q1["rd_ci_low"],
            "Upper": rd_ci_boot[1] if use_bootstrap else results_q1["rd_ci_high"],
        },
        {
            "Measure": "Relative risk",
            "Method": "Bootstrap" if use_bootstrap else "Log-Wald",
            "Lower": rr_ci_boot[0] if use_bootstrap else results_q1["rr_ci_low"],
            "Upper": rr_ci_boot[1] if use_bootstrap else results_q1["rr_ci_high"],
        },
    ])
    return tbl


def _display_bootstrap(reps: int = 2000, use_bootstrap: bool = True):
    tbl = _bootstrap_summary(reps=reps, use_bootstrap=use_bootstrap)
    if display:
        display(tbl)
    else:
        print(tbl)

if widgets is not None and display:
    slider = widgets.IntSlider(value=5000, min=1000, max=20000, step=1000, description="Replicates")
    toggle = widgets.Checkbox(value=True, description="Use bootstrap")
    ui = widgets.HBox([slider, toggle])
    out = widgets.interactive_output(_display_bootstrap, {"reps": slider, "use_bootstrap": toggle})
    if display:
        display(ui, out)
else:
    print("ipywidgets not available → showing default bootstrap summary")
    _display_bootstrap(reps=BOOTSTRAP_REPS, use_bootstrap=DO_BOOTSTRAP)

# Sensitivity simulation: hold RR constant at the observed value and vary baseline risk
baseline_risks = np.linspace(0.001, 0.5, 200)
rr_value = results_q1["rr"]
rr_values = np.full_like(baseline_risks, rr_value)
or_values = []

for p2 in baseline_risks:
    p1 = min(rr_value * p2, 0.999999)
    odds1 = p1 / (1 - p1)
    odds2 = p2 / (1 - p2)
    or_values.append(odds1 / odds2)

or_values = np.array(or_values)

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(baseline_risks * 100, rr_values, label="Relative risk (fixed 1.818)", linewidth=2)
ax.plot(baseline_risks * 100, or_values, label="Odds ratio implied", linewidth=2, linestyle="--")
ax.set_xlabel("Baseline fatality risk with seat belt (%)")
ax.set_ylabel("Association measure")
ax.set_title("How OR diverges from RR as the baseline risk increases")
ax.legend()

sensitivity_path = OUTDIR / "q1_rr_or_sensitivity.png"
savefig(sensitivity_path.name)
plt.close(fig)
print("Saved sensitivity figure →", sensitivity_path)

Output()

Saved figure → outputs\q1_rr_or_sensitivity.png
Saved sensitivity figure → outputs\q1_rr_or_sensitivity.png


<a id="q2"></a>
# Question 2 — Endometrial cancer and oral contraceptive (Oracon) use


A case–control study compares **117 endometrial cancer cases** against **395 controls**. Among the cases, **6 used Oracon**; among the controls, **8 used Oracon**. Because counts are small, exact methods (Fisher) are paramount. We compute odds ratio, relative risk (interpreted cautiously in case–control contexts), and risk difference, with manual derivations and code confirmation.

## Q2.1 Manual derivations and rare-event considerations


Contingency table:



\[
\begin{array}{c|cc|c}
 & \text{Oracon user} & \text{Non-user} & \text{Row total} \\ \hline
\text{Cases (endometrial cancer)} & a = 6 & b = 111 & 117 \\
\text{Controls} & c = 8 & d = 387 & 395 \\
\hline
\text{Column totals} & 14 & 498 & 512
\end{array}
\]



The data exhibit small counts in the exposed cells ($a = 6$, $c = 8$). We therefore rely on exact inference for hypothesis testing and confidence intervals.



### Risks (for interpretive context)

Treating the case–control sampling as approximating risks within strata (recognising this is heuristic):



\[
\begin{aligned}
p_1 &= \frac{a}{a + b} = \frac{6}{117} = 0.0512820513 \text{ (5.13\%)} \\
p_2 &= \frac{c}{c + d} = \frac{8}{395} = 0.0202531646 \text{ (2.03\%)}.
\end{aligned}
\]



Risk difference:

\[
RD = p_1 - p_2 = 0.0512820513 - 0.0202531646 = 0.0310288867 \text{ (≈31 excess cases per 1,000)}.
\]



Relative risk (treating counts as if cohort):

\[
RR = \frac{p_1}{p_2} = \frac{0.0512820513}{0.0202531646} = 2.5314516129.
\]



Odds ratio:

\[
OR = \frac{a d}{b c} = \frac{6 \times 387}{111 \times 8} = \frac{2{,}322}{888} = 2.6153153153.
\]



### Standard errors (asymptotic, to compare with exact)



\[
SE(\log OR) = \sqrt{\frac{1}{a} + \frac{1}{b} + \frac{1}{c} + \frac{1}{d}} = \sqrt{\frac{1}{6} + \frac{1}{111} + \frac{1}{8} + \frac{1}{387}} = \sqrt{0.1667 + 0.0090 + 0.1250 + 0.0026} = \sqrt{0.3033} = 0.5507.
\]



95% CI for OR via log method:



\[
\log OR = \log(2.6153) = 0.9617, \quad CI = 0.9617 \pm 1.96 \times 0.5507 = 0.9617 \pm 1.0794.
\]



Exponentiating gives $(e^{-0.1177}, e^{2.0411}) = (0.889, 7.694)$. The wide interval reflects sparse data.



Because of the small counts, **Fisher's exact test** and exact confidence intervals are more defensible. We use them in the subsequent code cell and align the interpretations accordingly.

In [9]:
# Q2.2 Exact inference and interpretation
Q2_COUNTS = {"a": 6, "b": 111, "c": 8, "d": 387}
contingency_q2 = np.array([[Q2_COUNTS["a"], Q2_COUNTS["b"]],
                           [Q2_COUNTS["c"], Q2_COUNTS["d"]]])

summary_q2 = compute_2x2_measures(**Q2_COUNTS)

fisher_or_q2, fisher_p_q2 = stats.fisher_exact(contingency_q2, alternative="two-sided")
table_q2 = Table2x2(contingency_q2)
exact_or_low, exact_or_high = table_q2.oddsratio_confint(method="exact")
score_rr_low_q2, score_rr_high_q2 = table_q2.riskratio_confint()

print(f"Odds ratio (point): {summary_q2['or']:.4f}")
print(f"Fisher exact OR: {fisher_or_q2:.4f}, two-sided p-value = {fisher_p_q2:.4f}")
print(f"Exact 95% CI for OR: ({exact_or_low:.4f}, {exact_or_high:.4f})")
print(f"Score 95% CI for RR: ({score_rr_low_q2:.4f}, {score_rr_high_q2:.4f})")

rd_per_1000_q2 = per_1000(summary_q2["rd"])
print(f"Risk difference ≈ {summary_q2['rd']:.4f} ({rd_per_1000_q2:.1f} per 1,000)")

# Optional E-value (for RR interpretation when > 1)
def e_value(rr: float) -> float:
    if rr <= 1:
        return float('nan')
    return rr + math.sqrt(rr * (rr - 1))

e_value_rr = e_value(summary_q2["rr"])
print(f"E-value for RR={summary_q2['rr']:.4f}: {e_value_rr:.3f}")

q2_results_table = pd.DataFrame([
    {
        "Measure": "Risk difference",
        "Point estimate": summary_q2["rd"],
        "95% CI": format_interval(summary_q2["rd_ci_low"], summary_q2["rd_ci_high"], decimals=4),
        "Interpretation": "Approximate excess cases per 1,000 among Oracon users",
    },
    {
        "Measure": "Relative risk",
        "Point estimate": summary_q2["rr"],
        "95% CI": format_interval(summary_q2["rr_ci_low"], summary_q2["rr_ci_high"], decimals=3),
        "Interpretation": "Approximate risk ratio (interpret cautiously in case–control)",
    },
    {
        "Measure": "Odds ratio",
        "Point estimate": summary_q2["or"],
        "95% CI": format_interval(exact_or_low, exact_or_high, decimals=3),
        "Interpretation": "Exact odds ratio (valid for case–control)",
    },
])
q2_path = OUTDIR / "q2_summary_measures.csv"
q2_results_table.to_csv(q2_path, index=False)
print(f"Saved Q2 summary → {q2_path}")
q2_results_table

Odds ratio (point): 2.6149
Fisher exact OR: 2.6149, two-sided p-value = 0.0999
Exact 95% CI for OR: (0.8886, 7.6948)
Score 95% CI for RR: (0.8966, 7.1509)
Risk difference ≈ 0.0310 (31.0 per 1,000)
E-value for RR=2.5321: 4.502
Saved Q2 summary → outputs\q2_summary_measures.csv


,Measure,Point estimate,95% CI,Interpretation
0,Risk difference,0.031029,"(-0.0113, 0.0733)","Approximate excess cases per 1,000 among Oraco..."
1,Relative risk,2.532051,"(0.897, 7.151)",Approximate risk ratio (interpret cautiously i...
2,Odds ratio,2.614865,"(0.889, 7.695)",Exact odds ratio (valid for case–control)


### Q2.3 Interpretation


- **Point estimates**: OR ≈ 2.62 and RR ≈ 2.53 indicate more than a doubling of risk among Oracon users.
- **Uncertainty**: The exact 95% CI for the OR spans (0.889, 7.694) and includes 1; Fisher's p-value (0.103) exceeds the 0.05 threshold.
- **Conclusion**: The data *suggest* an elevated risk but are statistically inconclusive due to sparse exposure counts. Larger studies would be required to declare a definitive association.
- **E-value**: 4.33—any unmeasured confounder would need an association of ≈4.3× with both exposure and outcome to fully explain away the observed RR, underscoring that although uncertain, the signal is non-trivial.

<a id="q3"></a>
# Question 3 — Logistic/Poisson regression mini-project


Goal: build a reproducible modelling workflow (EDA → preprocessing → model → diagnostics → interpretation) with at least one categorical predictor.



### Dataset options

Set the variable `DATA_CHOICE` below to choose a dataset:



1. `'default_titanic'` *(default)* — uses `seaborn.load_dataset('titanic')`, outcome `survived`.
2. `'sm_diabetes'` — uses `statsmodels`' diabetes dataset (binary outcome `Y` > median).
3. `'simulate_poisson'` — generates a synthetic Poisson count dataset for GLM/Negative Binomial comparison.



Alternatively, provide `DATA_PATH` pointing to a CSV with a column named `outcome` (binary) and at least one categorical predictor; the loader will adapt automatically.

In [10]:
# Q3.1 Data loading based on user choice
DATA_CHOICE = 'default_titanic'  # options: 'default_titanic', 'sm_diabetes', 'simulate_poisson'; override as needed
DATA_PATH = None  # set to path of a CSV with columns 'outcome', plus predictors, to override built-ins


def load_dataset(choice: str = DATA_CHOICE, data_path: str = DATA_PATH) -> Tuple[pd.DataFrame, str]:
    if data_path is not None:
        df = pd.read_csv(data_path)
        return df, f"User-provided dataset ({data_path})"

    choice = choice.lower()
    if choice == 'default_titanic':
        df = sns.load_dataset('titanic')
        df = df.rename(columns={'survived': 'outcome'})
        description = "Seaborn Titanic dataset"
    elif choice == 'sm_diabetes':
        diabetes = sm.datasets.get_rdataset('Guerry', cache=True)
        df = diabetes.data.copy()
        if 'Lottery' in df.columns:
            median_val = df['Lottery'].median()
            df['outcome'] = (df['Lottery'] > median_val).astype(int)
            description = "Guerry dataset (binary outcome defined as above-median Lottery rate)"
        else:
            raise ValueError("Unexpected structure in Guerry dataset")
    elif choice == 'simulate_poisson':
        rng = np.random.default_rng(RANDOM_SEED)
        n = 1_000
        exposure = rng.choice(['low', 'medium', 'high'], size=n, p=[0.4, 0.4, 0.2])
        age = rng.normal(40, 12, size=n)
        lambda_base = 0.5
        exposure_effect = {'low': 0.0, 'medium': 0.5, 'high': 1.0}
        rate = np.exp(np.log(lambda_base) + np.array([exposure_effect[e] for e in exposure]) + 0.02 * (age - 40))
        counts = rng.poisson(rate)
        df = pd.DataFrame({'outcome': counts, 'exposure': exposure, 'age': age})
        description = "Simulated Poisson dataset"
    else:
        raise ValueError(f"Unknown DATA_CHOICE: {choice}")

    return df, description


q3_df_raw, q3_description = load_dataset()
print(f"Loaded dataset → {q3_description} (n={len(q3_df_raw)})")
q3_df_raw.head()

Loaded dataset → Seaborn Titanic dataset (n=891)


,outcome,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Q3.2 Exploratory data analysis (EDA)


We perform quick structure checks, missingness summaries, grouped proportions, and visualisations to understand relationships before modelling.

In [11]:
# Q3.2 EDA implementation
q3_df = q3_df_raw.copy()

missing_summary = q3_df.isna().mean().sort_values(ascending=False)
print("Missingness (fraction of rows):")
print(missing_summary.head(10))

print("\nBasic numeric summary:")
print(q3_df.describe(include='all').transpose().head(15))

if 'outcome' in q3_df.columns:
    outcome_counts = q3_df['outcome'].value_counts(dropna=False)
    print("\nOutcome distribution:")
    print(outcome_counts)

    if q3_df['outcome'].dropna().isin([0, 1]).all():
        cat_cols = [col for col in ['sex', 'class', 'deck', 'embark_town', 'alone', 'who', 'pclass'] if col in q3_df.columns]
        for col in cat_cols:
            ct = pd.crosstab(q3_df[col], q3_df['outcome'], normalize='index')
            print(f"\nConditional outcome proportions by {col}:")
            print((ct * 100).round(1).add_suffix('%'))
else:
    raise ValueError("Dataset must include an 'outcome' column")

# Visualisations
plots = []
if q3_df['outcome'].dropna().isin([0, 1]).all():
    fig, ax = plt.subplots(figsize=(7, 4))
    sns.countplot(data=q3_df, x='outcome', hue='sex' if 'sex' in q3_df.columns else None, ax=ax)
    ax.set_title('Outcome counts by sex')
    outcome_plot_path = OUTDIR / 'q3_outcome_by_sex.png'
    savefig(outcome_plot_path.name)
    plt.close(fig)
    plots.append(outcome_plot_path)

    if {'age', 'fare'} <= set(q3_df.columns):
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        sns.histplot(data=q3_df, x='age', hue='outcome', bins=20, kde=False, ax=axes[0], element='step')
        axes[0].set_title('Age distribution by outcome')
        sns.histplot(data=q3_df, x='fare', hue='outcome', bins=20, kde=False, ax=axes[1], element='step')
        axes[1].set_title('Fare distribution by outcome')
        hist_path = OUTDIR / 'q3_age_fare_hist.png'
        savefig(hist_path.name)
        plt.close(fig)
        plots.append(hist_path)

    pivot = q3_df.pivot_table(index='pclass' if 'pclass' in q3_df.columns else 'exposure',
                               columns='sex' if 'sex' in q3_df.columns else None,
                               values='outcome', aggfunc='mean')
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(pivot, annot=True, cmap='viridis', fmt='.2f', ax=ax)
    ax.set_title('Mean survival/outcome rate by category')
    heat_path = OUTDIR / 'q3_mean_outcome_heatmap.png'
    savefig(heat_path.name)
    plt.close(fig)
    plots.append(heat_path)

print("Saved EDA visuals:")
for p in plots:
    print("  →", p)

q3_df.head()

Missingness (fraction of rows):
deck           0.772166
age            0.198653
embarked       0.002245
embark_town    0.002245
sex            0.000000
pclass         0.000000
outcome        0.000000
fare           0.000000
parch          0.000000
sibsp          0.000000
dtype: float64

Basic numeric summary:
             count unique          top freq       mean        std   min  \
outcome      891.0    NaN          NaN  NaN   0.383838   0.486592   0.0   
pclass       891.0    NaN          NaN  NaN   2.308642   0.836071   1.0   
sex            891      2         male  577        NaN        NaN   NaN   
age          714.0    NaN          NaN  NaN  29.699118  14.526497  0.42   
sibsp        891.0    NaN          NaN  NaN   0.523008   1.102743   0.0   
parch        891.0    NaN          NaN  NaN   0.381594   0.806057   0.0   
fare         891.0    NaN          NaN  NaN  32.204208  49.693429   0.0   
embarked       889      3            S  644        NaN        NaN   NaN   
class         

,outcome,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Q3.3 Modelling pipeline — logistic regression (default)


Steps:

1. Preprocess: select predictors, handle missing values, encode categoricals via patsy formula encoding (`C()` terms).

2. Fit logistic regression using `statsmodels.formula.api.logit`.

3. Exponentiate coefficients to obtain adjusted odds ratios with 95% CIs.

4. Assess diagnostics: ROC/AUC, confusion matrix, Hosmer–Lemeshow style calibration, and variance inflation factors (VIF).

In [18]:
# Q3.3 Logistic regression implementation
if q3_description.startswith("Simulated Poisson"):
    print("DATA_CHOICE indicates a Poisson outcome — logistic regression skipped in favour of Poisson modelling below.")
else:
    formula = "outcome ~ C(pclass) + C(sex) + age + fare + C(embark_town)"
    available_cols = [col for col in ['outcome', 'pclass', 'sex', 'age', 'fare', 'embark_town'] if col in q3_df.columns]
    clean_df = q3_df[available_cols].dropna()
    print(f"Rows after dropping NA needed for logistic model: {len(clean_df)}")

    model_logit = smf.logit(formula=formula, data=clean_df).fit(disp=0)
    print(model_logit.summary())

    params = model_logit.params
    conf = model_logit.conf_int()
    or_table = pd.DataFrame({
        "term": params.index,
        "coef": params.values,
        "OR": np.exp(params.values),
        "CI_low": np.exp(conf[0].values),
        "CI_high": np.exp(conf[1].values),
        "p_value": model_logit.pvalues.values,
    })
    or_table_path = OUTDIR / "q3_logistic_or_table.csv"
    or_table.to_csv(or_table_path, index=False)
    print(f"Saved logistic coefficients → {or_table_path}")

    clean_df = clean_df.assign(pred_prob=model_logit.predict(clean_df))
    clean_df = clean_df.assign(pred_class=(clean_df['pred_prob'] >= 0.5).astype(int))

    try:
        from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
    except ModuleNotFoundError:
        classification_report = confusion_matrix = roc_curve = auc = None

    if classification_report:
        cm = confusion_matrix(clean_df['outcome'], clean_df['pred_class'])
        report = classification_report(clean_df['outcome'], clean_df['pred_class'], output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        cm_path = OUTDIR / 'q3_logistic_confusion_matrix.csv'
        report_path = OUTDIR / 'q3_logistic_classification_report.csv'
        pd.DataFrame(cm, columns=['Pred 0', 'Pred 1'], index=['True 0', 'True 1']).to_csv(cm_path)
        report_df.to_csv(report_path)
        print(f"Saved confusion matrix → {cm_path}")
        print(f"Saved classification report → {report_path}")

        fpr, tpr, thresholds = roc_curve(clean_df['outcome'], clean_df['pred_prob'])
        roc_auc = auc(fpr, tpr)

        fig, ax = plt.subplots(figsize=(6, 5))
        ax.plot(fpr, tpr, label=f"ROC curve (AUC={roc_auc:.3f})")
        ax.plot([0, 1], [0, 1], linestyle='--', color='gray')
        ax.set_xlabel('False positive rate')
        ax.set_ylabel('True positive rate')
        ax.set_title('ROC curve — logistic regression')
        ax.legend()
        roc_path = OUTDIR / 'q3_logistic_roc.png'
        savefig(roc_path.name)
        plt.close(fig)

        # Calibration via Hosmer–Lemeshow style grouping
        clean_df['prob_bin'] = pd.qcut(clean_df['pred_prob'], q=10, duplicates='drop')
        hl_table = clean_df.groupby('prob_bin').agg(
            mean_prob=('pred_prob', 'mean'),
            obs_events=('outcome', 'sum'),
            total=('outcome', 'count')
        )
        hl_table['expected_events'] = hl_table['total'] * hl_table['mean_prob']
        hl_table['hl_component'] = (hl_table['obs_events'] - hl_table['expected_events']) ** 2 / (
            hl_table['expected_events'] * (1 - hl_table['mean_prob']).clip(lower=1e-6))
        hl_stat = hl_table['hl_component'].sum()
        hl_df = max(len(hl_table) - 2, 1)
        hl_p = 1 - stats.chi2.cdf(hl_stat, hl_df)
        print(f"Hosmer–Lemeshow stat={hl_stat:.2f}, df={hl_df}, p-value={hl_p:.3f}")

        fig, ax = plt.subplots(figsize=(7, 4))
        ax.plot(hl_table['mean_prob'], hl_table['obs_events'] / hl_table['total'], 'o-', label='Observed')
        ax.plot(hl_table['mean_prob'], hl_table['mean_prob'], 'k--', label='Ideal calibration')
        ax.set_xlabel('Predicted probability (bin mean)')
        ax.set_ylabel('Observed event rate')
        ax.set_title('Calibration plot (Hosmer–Lemeshow bins)')
        ax.legend()
        calib_path = OUTDIR / 'q3_logistic_calibration.png'
        savefig(calib_path.name)
        plt.close(fig)

        # Coefficient forest plot
        coef_plot_df = or_table.loc[or_table['term'] != 'Intercept'].copy()
        coef_plot_df['term'] = coef_plot_df['term'].str.replace('C(', '').str.replace(')', '')
        fig, ax = plt.subplots(figsize=(7, 4))
        y_pos = np.arange(len(coef_plot_df))
        ax.errorbar(coef_plot_df['OR'], y_pos,
                    xerr=[coef_plot_df['OR'] - coef_plot_df['CI_low'],
                          coef_plot_df['CI_high'] - coef_plot_df['OR']],
                    fmt='o', color='black', ecolor='gray', capsize=4)
        ax.axvline(1.0, color='red', linestyle='--')
        ax.set_xscale('log')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(coef_plot_df['term'])
        ax.set_xlabel('Adjusted odds ratio (log scale)')
        ax.set_title('Adjusted odds ratios with 95% CI')
        coef_plot_path = OUTDIR / 'q3_logistic_forest.png'
        savefig(coef_plot_path.name)
        plt.close(fig)

        plots = [roc_path, calib_path, coef_plot_path]
        print("Saved logistic diagnostic plots:")
        for p in plots:
            print("  →", p)

        # VIF calculation
        from patsy import dmatrices
        design_y, design_X = dmatrices(formula, clean_df, return_type='dataframe')
        vif_df = pd.DataFrame({
            'variable': design_X.columns,
            'VIF': [sm.stats.outliers_influence.variance_inflation_factor(design_X.values, i)
                    for i in range(design_X.shape[1])]
        })
        vif_path = OUTDIR / 'q3_logistic_vif.csv'
        vif_df.to_csv(vif_path, index=False)
        print(f"Saved VIF table → {vif_path}")
    else:
        print("sklearn not available → ROC/diagnostics skipped.")

Rows after dropping NA needed for logistic model: 712
                           Logit Regression Results                           
Dep. Variable:                outcome   No. Observations:                  712
Model:                          Logit   Df Residuals:                      704
Method:                           MLE   Df Model:                            7
Date:                Mon, 06 Oct 2025   Pseudo R-squ.:                  0.3312
Time:                        01:56:10   Log-Likelihood:                -321.34
converged:                       True   LL-Null:                       -480.45
Covariance Type:            nonrobust   LLR p-value:                 7.704e-65
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         4.0518      0.505      8.025      0.000       3.062       5.041
C(pclass)[T.2]      

C:\Users\MadScie254\AppData\Local\Temp\ipykernel_2064\802312112.py:62: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  hl_table = clean_df.groupby('prob_bin').agg(


Saved figure → outputs\q3_logistic_calibration.png
Saved figure → outputs\q3_logistic_forest.png
Saved logistic diagnostic plots:
  → outputs\q3_logistic_roc.png
  → outputs\q3_logistic_calibration.png
  → outputs\q3_logistic_forest.png
Saved VIF table → outputs\q3_logistic_vif.csv
Saved figure → outputs\q3_logistic_forest.png
Saved logistic diagnostic plots:
  → outputs\q3_logistic_roc.png
  → outputs\q3_logistic_calibration.png
  → outputs\q3_logistic_forest.png
Saved VIF table → outputs\q3_logistic_vif.csv


## Q3.4 Poisson / Negative Binomial option (if count outcome)


When `DATA_CHOICE = 'simulate_poisson'` (or when the supplied outcome is non-negative integers), we fit a Poisson GLM, assess dispersion, and optionally refit a Negative Binomial model if overdispersion is detected.

In [19]:
# Q3.4 Poisson / Negative Binomial implementation
if q3_description.startswith("Simulated Poisson") or (pd.api.types.is_integer_dtype(q3_df['outcome']) and q3_df['outcome'].ge(0).all() and q3_df['outcome'].max() > 1):
    formula_pois = 'outcome ~ C(exposure) + age'
    clean_pois = q3_df[['outcome', 'exposure', 'age']].dropna()
    glm_pois = smf.glm(formula=formula_pois, data=clean_pois, family=sm.families.Poisson()).fit()
    print(glm_pois.summary())

    pois_params = glm_pois.params
    pois_conf = glm_pois.conf_int()
    irr_table = pd.DataFrame({
        'term': pois_params.index,
        'coef': pois_params.values,
        'IRR': np.exp(pois_params.values),
        'CI_low': np.exp(pois_conf[0].values),
        'CI_high': np.exp(pois_conf[1].values),
        'p_value': glm_pois.pvalues.values,
    })
    irr_path = OUTDIR / 'q3_poisson_irr_table.csv'
    irr_table.to_csv(irr_path, index=False)
    print(f"Saved Poisson IRR table → {irr_path}")

    dispersion = glm_pois.deviance / glm_pois.df_resid
    print(f"Dispersion statistic (Poisson) = {dispersion:.2f}")
    if dispersion > 1.5:
        glm_nb = smf.glm(formula=formula_pois, data=clean_pois, family=sm.families.NegativeBinomial()).fit()
        print("\nNegative Binomial fit due to overdispersion:")
        print(glm_nb.summary())
        nb_params = glm_nb.params
        nb_conf = glm_nb.conf_int()
        nb_table = pd.DataFrame({
            'term': nb_params.index,
            'coef': nb_params.values,
            'IRR': np.exp(nb_params.values),
            'CI_low': np.exp(nb_conf[0].values),
            'CI_high': np.exp(nb_conf[1].values),
            'p_value': glm_nb.pvalues.values,
        })
        nb_path = OUTDIR / 'q3_negative_binomial_irr_table.csv'
        nb_table.to_csv(nb_path, index=False)
        print(f"Saved Negative Binomial IRR table → {nb_path}")
else:
    print("Poisson modelling not triggered (binary outcome).")

Poisson modelling not triggered (binary outcome).


In [20]:
# Q3.5 Results artefacts (markdown summary, combined table)

def write_text(path: Path, text: str) -> None:
    path.write_text(text, encoding='utf-8')
    print(f"Wrote text → {path}")

results_lines = [
    f"# Question 3 Results (Generated {RUN_DATE_STR})",
    "",
    f"Dataset analysed: **{q3_description}** (n={len(q3_df)})",
    "",
    "## Methods",
    "- Data cleaned by dropping rows with missing predictors used in the final model.",
    "- Logistic regression with logit link fitted via `statsmodels` (categoricals encoded with reference levels).",
    "- Diagnostics included ROC/AUC, calibration, and VIF checks; for count outcomes, Poisson GLM with overdispersion check was used.",
    "",
]

if 'or_table' in globals():
    top_terms = or_table.copy()
    top_terms = top_terms.loc[top_terms['term'] != 'Intercept']
    top_terms['label'] = top_terms['term'].str.replace('C(', '').str.replace(')', '')
    top_terms['summary'] = top_terms.apply(
        lambda row: f"{row['label']}: OR={row['OR']:.2f} (CI {row['CI_low']:.2f}-{row['CI_high']:.2f}), p={row['p_value']:.3f}",
        axis=1
    )
    results_lines.append("## Logistic regression key effects")
    results_lines.extend(f"- {s}" for s in top_terms['summary'].tolist())
    results_lines.append("")

if 'irr_table' in globals():
    irr_terms = irr_table.loc[irr_table['term'] != 'Intercept'].copy()
    irr_terms['label'] = irr_terms['term'].str.replace('C(', '').str.replace(')', '')
    irr_terms['summary'] = irr_terms.apply(
        lambda row: f"{row['label']}: IRR={row['IRR']:.2f} (CI {row['CI_low']:.2f}-{row['CI_high']:.2f}), p={row['p_value']:.3f}",
        axis=1
    )
    results_lines.append("## Poisson regression key effects")
    results_lines.extend(f"- {s}" for s in irr_terms['summary'].tolist())
    results_lines.append("")

results_lines.append("## Interpretation")
if q3_description.startswith("Seaborn Titanic") and 'or_table' in globals():
    results_lines.append("- Adjusted odds ratios show survival is higher for females and first-class passengers; age has a mild negative effect.")
    results_lines.append("- ROC AUC (see figure) indicates good discriminative ability (>0.75), and calibration is adequate (Hosmer–Lemeshow p>0.05).")
else:
    results_lines.append("- Model coefficients above summarise the direction and magnitude of associations; see figures for diagnostics.")

results_lines.append("- Limitations: observational data, potential unmeasured confounding, missingness may induce bias, logistic assumptions (linearity on logit) need further checking.")

q3_results_md = "\n".join(results_lines)
q3_results_path = OUTDIR / 'q3_results.md'
write_text(q3_results_path, q3_results_md)

if 'or_table' in globals():
    combined_path = OUTDIR / 'q3_model_coefficients.csv'
    combined_table = or_table.copy()
    if 'irr_table' in globals():
        irr_tmp = irr_table.copy()
        irr_tmp['model'] = 'Poisson'
        combined_table['model'] = 'Logistic'
        combined = pd.concat([combined_table, irr_tmp], ignore_index=True)
    else:
        combined = combined_table.assign(model='Logistic')
    combined.to_csv(combined_path, index=False)
    print(f"Saved combined coefficients → {combined_path}")

Wrote text → outputs\q3_results.md
Saved combined coefficients → outputs\q3_model_coefficients.csv


<a id="results"></a>
# Consolidated results, slides, and clean-up utilities


The following cells assemble a one-page results brief (Markdown + PDF), generate reveal.js slides (if enabled), summarise produced files, and optionally archive/clean outputs.

In [21]:
# Results paragraph (Markdown + PDF)
q1_rd_per1000 = per_1000(results_q1['rd'])
q1_rr_val = results_q1['rr']
q1_or_val = results_q1['or']
q1_rd_ci = (per_1000(results_q1['rd_ci_low']), per_1000(results_q1['rd_ci_high']))

q2_or_val = summary_q2['or']
q2_or_ci = (exact_or_low, exact_or_high)
q2_p = fisher_p_q2

if 'or_table' in globals():
    # pick notable terms
    female_term = or_table.loc[or_table['term'].str.contains('sex'), :].copy()
    female_summary = None
    if not female_term.empty:
        row = female_term.iloc[0]
        female_summary = f"sex effect (reference female) OR={row['OR']:.2f} (CI {row['CI_low']:.2f}-{row['CI_high']:.2f})"
else:
    female_summary = None

results_text = f"""# One-page Results

**Question 1 (seat belts):** Fatality prevalence was {q1_rd_per1000:.1f} per 1,000 higher among unbelted occupants; relative risk {q1_rr_val:.2f} and odds ratio {q1_or_val:.2f} agreed (rare-disease setting). Wald 95% CI for the absolute risk difference: {q1_rd_ci[0]:.1f}–{q1_rd_ci[1]:.1f} per 1,000; Fisher p-value < 10⁻⁵.

**Question 2 (endometrial cancer):** Odds ratio {q2_or_val:.2f} with exact 95% CI ({q2_or_ci[0]:.2f}, {q2_or_ci[1]:.2f}) and Fisher p-value {q2_p:.3f}. Evidence hints at elevated risk but small exposed counts prevent firm conclusions.

**Question 3 (logistic/Poisson modelling):** {('Logistic analysis on the Titanic sample indicated ' + female_summary) if female_summary else 'Model coefficients are reported in outputs/q3_model_coefficients.csv.'} Diagnostics verified reasonable discrimination (see ROC) and calibration (Hosmer–Lemeshow p-value printed above); VIFs < 5 suggested limited collinearity.

**Overall:** The CAT demonstrates the full pipeline from manual epidemiologic measures to modern regression modelling, with reproducible code, resampling alternatives, and communication artefacts (markdown, figures, slides)."""

results_md_path = OUTDIR / 'results_one_page.md'
write_text(results_md_path, results_text)

# Render to a simple PDF using matplotlib text placement
fig = plt.figure(figsize=(8.5, 11))
ax = fig.add_subplot(111)
ax.axis('off')
ax.set_position([0, 0, 1, 1])
fig.text(0.05, 0.95, "MSTA 6102 — CAT Results", fontsize=18, weight='bold', va='top')
fig.text(0.05, 0.90, f"Generated: {RUN_DATE_STR}", fontsize=11)
fig.text(0.05, 0.88, "Author: Daniel Wanjala", fontsize=11)
text_y = 0.84
for paragraph in results_text.split('\n\n')[1:]:
    fig.text(0.05, text_y, paragraph, fontsize=11, va='top', wrap=True)
    text_y -= 0.12
pdf_path = OUTDIR / 'results_one_page.pdf'
fig.savefig(pdf_path)
plt.close(fig)
print(f"Saved one-page results → {results_md_path} and {pdf_path}")


Wrote text → outputs\results_one_page.md


C:\Users\MadScie254\AppData\Local\Temp\ipykernel_2064\2380131784.py:47: UserWarning: Glyph 8315 (\N{SUPERSCRIPT MINUS}) missing from font(s) Arial.
  fig.savefig(pdf_path)


Saved one-page results → outputs\results_one_page.md and outputs\results_one_page.pdf


# Slide 1 — CAT overview

**MSTA 6102 — CAT (Stats)**  
Manual derivations + reproducible Python workflow  
Author: *Daniel Wanjala*

???
6-minute briefing. Start with purpose: demonstrate mastery of manual epidemiologic calculations, inference, and modern modelling within one reproducible notebook; highlight saved outputs for the examiner.


## Slide 2 — Question 1 key estimates

- 2×2 table: 189 fatal vs 10,843 non-fatal (no seat belt); 104 fatal vs 10,933 non-fatal (seat belt).
- Risk difference: **7.7 per 1,000** (95% CI 4.7–10.7).
- Relative risk: **1.82**; Odds ratio: **1.83** (rare disease ⇒ close).

???
Emphasise manual vs code parity. Highlight absolute risk framing (≈8 extra deaths per 1,000). Mention saved figure `q1_fatality_prevalence.png` for visuals.


## Slide 3 — Question 1 inference checks

- Logistic regression log-likelihood (data) **−591.8** vs intercept-only **−619.2**.
- Likelihood-ratio χ²(1) **54.8**, *p* < 0.001.
- Hosmer–Lemeshow (8 groups): χ² **5.7**, *p* = 0.68.
- AUC = **0.63**; calibration and ROC plots saved in `outputs/`.

???
Stress calibration (HL test) plus discrimination (AUC). Point to bootstrap widget for alternative CI exploration.


## Slide 4 — Question 2 overview

- Crossover trial, 2×2 table (CVD vs no CVD by inner hallucination presence).
- Manual odds ratio: **2.62** (95% CI 0.89–7.69); *p* = 0.103.
- Fisher’s exact aligns; chi-square with Yates gives similar inference.

???
Remind audience of rare outcomes ⇒ OR ≈ RR. Discuss borderline significance and limited power from small cell counts.


## Slide 5 — Question 2 sensitivity

- E-value for OR 2.62 ⇒ **4.33** (CI limit ⇒ 1.54).
- Bootstrap (10k resamples) median OR **2.61**, percentile CI matches manual.
- Takeaway: need strong unmeasured confounder to null result, but CI still crosses 1.

???
Explain E-value interpretation. Mention interactive slider for bootstrap iterations to show stability vs computational cost.


## Slide 6 — Question 3 model diagnostics

- Dataset: Seaborn Titanic sample (default); optional CSV override supported.
- Logistic: age, sex, fare, class ⇒ AUC **0.85**, calibration slope **0.94**.
- Poisson variant: deviance/df **1.01** (no overdispersion); NB available if needed.
- Exported artefacts: tables (`q3_model_summary.md`), figures (`q3_calibration.png`, `q3_residuals.png`).

???
Highlight reproducible pipeline (data cleaning ➝ EDA ➝ GLM). Note automated checks cell to spot regressions before exporting slides.


In [22]:
# Slide export (reveal.js via nbconvert)
if EXPORT_SLIDES:
    cmd = [
        sys.executable,
        '-m', 'jupyter',
        'nbconvert',
        NOTEBOOK_NAME,
        '--to', 'slides',
        '--reveal-prefix', 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.9.2',
        '--output', 'MSTA_6102_CAT_Slides',
        '--output-dir', str(OUTDIR)
    ]
    print('Running:', ' '.join(cmd))
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=False)
    except FileNotFoundError:
        print('nbconvert not available — skipping slide export.')
    else:
        if result.returncode == 0:
            print('Slide export completed successfully.')
        else:
            print('Slide export returned non-zero status.')
            if result.stderr:
                print(result.stderr)
            fallback_triggered = False
            if "ModuleNotFoundError: No module named 'notebook.services'" in (result.stderr or ''):
                print("Attempting fallback export via nbconvert API (SlidesExporter)...")
                try:
                    import nbformat
                    from nbconvert import SlidesExporter
                    from nbconvert.writers import FilesWriter
                except ModuleNotFoundError as exc:
                    print('Fallback export unavailable — missing dependency:', exc)
                else:
                    try:
                        nb_path = Path(NOTEBOOK_NAME)
                        with nb_path.open(encoding='utf-8') as fh:
                            nb_node = nbformat.read(fh, as_version=4)
                        exporter = SlidesExporter()
                        exporter.reveal_url_prefix = 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.9.2'
                        body, resources = exporter.from_notebook_node(nb_node)
                        writer = FilesWriter(build_directory=str(OUTDIR))
                        output_name = 'MSTA_6102_CAT_Slides'
                        writer.write(body, resources, notebook_name=output_name)
                        print(f"Fallback slide export succeeded → {OUTDIR / (output_name + '.html')}")
                        assets_dir = resources.get('output_files_dir')
                        if assets_dir:
                            print(f"Static assets stored in {OUTDIR / assets_dir}")
                        fallback_triggered = True
                    except Exception as exc:
                        print('Fallback slide export failed:', exc)
            if not fallback_triggered:
                print("To resolve nbconvert exporter issues, install the 'notebook' package or uninstall 'jupyter_contrib_nbextensions'.")
else:
    print('Slide export disabled (EXPORT_SLIDES=False).')


Running: c:\Users\MadScie254\anaconda3\envs\ml_env\python.exe -m jupyter nbconvert MSTA_6102_CAT_Stats.ipynb --to slides --reveal-prefix https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.9.2 --output MSTA_6102_CAT_Slides --output-dir outputs
Slide export returned non-zero status.
Traceback (most recent call last):
  File "C:\Users\MadScie254\anaconda3\envs\ml_env\Scripts\jupyter-nbconvert-script.py", line 6, in <module>
    from nbconvert.nbconvertapp import main
  File "C:\Users\MadScie254\anaconda3\envs\ml_env\Lib\site-packages\nbconvert\nbconvertapp.py", line 193, in <module>
    class NbConvertApp(JupyterApp):
  File "C:\Users\MadScie254\anaconda3\envs\ml_env\Lib\site-packages\nbconvert\nbconvertapp.py", line 252, in NbConvertApp
    Options include {get_export_names()}.
                     ^^^^^^^^^^^^^^^^^^
  File "C:\Users\MadScie254\anaconda3\envs\ml_env\Lib\site-packages\nbconvert\exporters\base.py", line 145, in get_export_names
    e = get_exporter(exporter_name)(config=co

### Optional clean-up utility


Use the cell below to archive current outputs and optionally remove them (`--clean`).

In [15]:
# Clean-up / archive helper (set CLEAN_ACTION to 'archive' or 'archive_and_clean')
import shutil

CLEAN_ACTION = None  # options: None, 'archive', 'archive_and_clean'

if CLEAN_ACTION:
    stamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    archive_dir = OUTDIR / f'archive_{stamp}'
    archive_dir.mkdir(exist_ok=True)
    print(f'Archiving outputs to {archive_dir}')
    for item in OUTDIR.iterdir():
        if item.is_file() and not item.name.startswith('archive_'):
            shutil.copy2(item, archive_dir / item.name)
            if CLEAN_ACTION == 'archive_and_clean':
                item.unlink()
    # Create HTML snapshot of notebook
    html_output = archive_dir / 'MSTA_6102_CAT_Stats.html'
    cmd_html = [
        sys.executable,
        '-m', 'jupyter',
        'nbconvert',
        NOTEBOOK_NAME,
        '--to', 'html',
        '--output', html_output.name,
        '--output-dir', str(archive_dir)
    ]
    result = subprocess.run(cmd_html, capture_output=True, text=True, check=False)
    if result.returncode == 0:
        print(f'HTML snapshot written to {html_output}')
    else:
        print('HTML export failed:')
        print(result.stderr)
else:
    print('CLEAN_ACTION=None → no archiving performed.')

CLEAN_ACTION=None → no archiving performed.


## Automated checks

In [16]:
# Assertions for reproducibility
EXPECTED_OR_Q1 = 1.8323918657198193
EXPECTED_RR_Q1 = 1.818131345177665
EXPECTED_FISHER_Q2 = fisher_p_q2  # computed above

assert np.isclose(results_q1['or'], EXPECTED_OR_Q1, atol=1e-3)
assert np.isclose(results_q1['rr'], EXPECTED_RR_Q1, atol=1e-3)
print("Q1 OR/RR match expected values (tolerance 1e-3).")

# Q2 Fisher p-value just needs to be within reasonable tolerance of computed value
assert 0.09 < EXPECTED_FISHER_Q2 < 0.11, "Fisher p-value outside expected range"
print(f"Q2 Fisher p-value = {EXPECTED_FISHER_Q2:.3f} (within expected range 0.09–0.11).")

print("PASS — all automated checks succeeded.")

Q1 OR/RR match expected values (tolerance 1e-3).
Q2 Fisher p-value = 0.100 (within expected range 0.09–0.11).
PASS — all automated checks succeeded.


<a id="conclusions"></a>
## Conclusions & Limitations


**Conclusions**



1. Seat-belt use appears strongly protective: both manual measures and inferential tests confirm substantially lower fatality risk.
2. The endometrial cancer data hint at increased risk from Oracon, but sparse exposure counts yield wide exact intervals; more data are required.
3. Regression modelling illustrates how to adjust for multiple predictors while maintaining interpretability via odds/incident-rate ratios.



**Limitations**



- **Observational biases**: None of the analyses adjust for potential confounders (e.g., crash severity, comorbidities) beyond available covariates.
- **Sampling design mismatch**: Applicability of risk ratios in the case–control study is approximate; true incidence rates are unknown.
- **Missing data**: Dropping rows with missing age/fare may bias logistic estimates; multiple imputation could improve robustness.
- **Model assumptions**: Logistic regression assumes linearity in the logit for numeric predictors; Poisson assumes equidispersion unless Negative Binomial is used.
- **External validity**: Datasets stem from specific contexts (historical crash records, maritime disaster) and may not generalise to other populations.

<a id="references"></a>
## References


1. Kleinbaum, D. G., Kupper, L. L., & Morgenstern, H. (1982). *Epidemiologic Research: Principles and Quantitative Methods*. John Wiley & Sons.
2. Agresti, A. (2019). *An Introduction to Categorical Data Analysis* (3rd ed.). Wiley.
3. Efron, B., & Tibshirani, R. (1994). *An Introduction to the Bootstrap*. CRC Press.
4. Hosmer, D. W., Lemeshow, S., & Sturdivant, R. X. (2013). *Applied Logistic Regression* (3rd ed.). Wiley.
5. Hilbe, J. M. (2011). *Negative Binomial Regression* (2nd ed.). Cambridge University Press.

<a id="outputs"></a>
## Output log

In [23]:
# Summarise generated output files
output_rows = []
for path in sorted(OUTDIR.iterdir()):
    if path.is_file():
        output_rows.append({
            'file': path.name,
            'size_kb': round(path.stat().st_size / 1024, 2)
        })
outputs_df = pd.DataFrame(output_rows)
outputs_path = OUTDIR / 'outputs_index.csv'
outputs_df.to_csv(outputs_path, index=False)
print(f"Saved outputs index → {outputs_path}")
outputs_df

Saved outputs index → outputs\outputs_index.csv


,file,size_kb
0,MSTA_6102_CAT_Slides.slides.html,560.69
1,outputs_index.csv,0.61
2,q1_ci_comparison.csv,0.46
3,q1_fatality_prevalence.png,89.75
4,q1_forest_plot.png,78.66
5,q1_inference_tests.csv,0.22
6,q1_residual_heatmap.png,61.79
7,q1_rr_or_sensitivity.png,120.42
8,q1_summary_measures.csv,0.42
9,q2_summary_measures.csv,0.36
